In [1]:
import pandas as pd
import numpy as np
import xgboost as xgb
import warnings
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
warnings.filterwarnings("ignore")
%cd D:\Resources\Datasets\深圳北站周边交通拥堵指数

D:\Resources\Datasets\深圳北站周边交通拥堵指数


In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor

In [3]:
df1 = pd.read_csv('train_TTI_6+6+3_more_drop.csv')
df1['date'] = pd.DatetimeIndex(df1['time']).date
df1['time'] = pd.DatetimeIndex(df1['time']).time
df1.set_index(['id_road', 'time','date'], inplace=True)
df1.sort_index(inplace=True)
df2 = pd.concat([pd.read_csv('toPredict_noLabel.csv'), pd.read_csv('toPredict_train_TTI.csv')])
df2['time'] = pd.DatetimeIndex(df2['time'])
df3 = pd.DataFrame(columns=['id_sample', 'TTI'])
df1.corr()

,TTI0,TTI1,TTI2,TTI3,TTI4,TTI5,speed0,speed1,speed2,speed3,speed4,speed5,TTI6,TTI7,TTI8,isHoliday,weekday
TTI0,1.000000,0.913943,0.825248,0.762461,0.715100,0.674471,-0.646792,-0.617892,-0.585991,-0.559919,-0.537530,-0.517653,0.636325,0.598135,0.559219,-0.051272,-0.008241
TTI1,0.913943,1.000000,0.914789,0.826398,0.763876,0.715793,-0.621196,-0.649624,-0.620813,-0.588929,-0.562889,-0.540310,0.675446,0.636764,0.596449,-0.052615,-0.008486
TTI2,0.825248,0.914789,1.000000,0.914602,0.826465,0.763680,-0.589862,-0.622021,-0.650725,-0.621969,-0.590149,-0.564030,0.715736,0.674705,0.633910,-0.053274,-0.008279
TTI3,0.762461,0.826398,0.914602,1.000000,0.914404,0.826163,-0.562989,-0.589959,-0.622274,-0.651186,-0.622446,-0.590596,0.763292,0.714722,0.672087,-0.053408,-0.007883
TTI4,0.715100,0.763876,0.826465,0.914404,1.000000,0.914370,-0.538843,-0.562997,-0.590085,-0.622564,-0.651559,-0.622814,0.826007,0.762357,0.712298,-0.053680,-0.007840
TTI5,0.674471,0.715793,0.763680,0.826163,0.914370,1.000000,-0.516412,-0.538482,-0.562764,-0.589994,-0.622539,-0.651592,0.914214,0.825035,0.760211,-0.053786,-0.007731
speed0,-0.646792,-0.621196,-0.589862,-0.562989,-0.538843,-0.516412,1.000000,0.978521,0.956265,0.937033,0.919127,0.902554,-0.494271,-0.471746,-0.447997,0.070694,0.020180
speed1,-0.617892,-0.649624,-0.622021,-0.589959,-0.562997,-0.538482,0.978521,1.000000,0.978522,0.956274,0.937068,0.919153,-0.515670,-0.493066,-0.468666,0.071230,0.020310
speed2,-0.585991,-0.620813,-0.650725,-0.622274,-0.590085,-0.562764,0.956265,0.978522,1.000000,0.978519,0.956290,0.937076,-0.537822,-0.514480,-0.489845,0.071617,0.020318
speed3,-0.559919,-0.588929,-0.621969,-0.651186,-0.622564,-0.589994,0.937033,0.956274,0.978519,1.000000,0.978524,0.956287,-0.562210,-0.536719,-0.511325,0.071838,0.020253


In [10]:
df1

TTI0     TTI1     TTI2     TTI3     TTI4  \
id_road time     date                                                      
275911  00:00:00 2019-01-02  1.14105  1.13699  1.12393  1.15026  1.13117   
                 2019-01-03  1.53843  1.29012  1.16313  1.13825  1.37852   
                 2019-01-04  1.20683  1.29577  1.24171  1.15601  1.14226   
                 2019-01-05  1.58394  1.47470  1.25195  1.15187  1.16729   
                 2019-01-06  2.32614  1.51375  1.22063  1.20102  1.20443   
...                              ...      ...      ...      ...      ...   
276738  23:50:00 2019-12-16  1.28309  1.36590  1.37705  1.38399  1.20483   
                 2019-12-17  1.21852  1.42880  1.36848  1.26464  1.25062   
                 2019-12-18  1.39183  1.42769  1.41937  1.32931  1.29322   
                 2019-12-19  1.26962  1.34787  1.25425  1.27220  1.23676   
                 2019-12-20  1.35850  1.50126  1.84028  1.62766  1.31815   

                                TTI5   speed0   speed1   speed2   speed3  \
id_road time     date                                                      
275911  00:00:00 2019-01-02  1.11357  63.1037  63.3287  64.0646  62.5981   
                 2019-01-03  1.91348  46.8037  55.8121  61.9055  63.2586   
                 2019-01-04  1.15313  59.6641  55.5687  57.9878  62.2869   
                 2019-01-05  1.18005  45.4591  48.8351  57.5137  62.5106   
                 2019-01-06  1.12525  30.9755  47.5669  58.9894  59.9527   
...                              ...      ...      ...      ...      ...   
276738  23:50:00 2019-12-16  1.34748  31.9496  30.0126  29.7696  29.6202   
                 2019-12-17  1.32827  33.6425  28.6912  29.9559  32.4157   
                 2019-12-18  1.17574  29.4534  28.7136  28.8818  30.8386   
                 2019-12-19  1.27166  32.2884  30.4140  32.6842  32.2230   
                 2019-12-20  1.28073  30.1759  27.3064  22.2760  25.1859   

                              speed4   speed5     TTI6     TTI7     TTI8  \
id_road time     date                                                      
275911  00:00:00 2019-01-02  63.6550  64.6607  1.09580  1.10014  1.11471   
                 2019-01-03  52.2333  37.6436  2.08315  2.04741  2.08087   
                 2019-01-04  63.0367  62.4427  1.15265  1.30350  1.86703   
                 2019-01-05  61.6852  61.0182  1.15340  1.14601  1.15873   
                 2019-01-06  59.7828  63.9898  1.16382  1.16347  1.14915   
...                              ...      ...      ...      ...      ...   
276738  23:50:00 2019-12-16  34.0249  30.4227  1.42428  1.43791  1.36372   
                 2019-12-17  32.7790  30.8628  1.24253  1.26758  1.31894   
                 2019-12-18  31.6993  34.8667  1.28191  1.30337  1.18777   
                 2019-12-19  33.1464  32.2367  1.22254  1.28557  1.17445   
                 2019-12-20  31.0996  32.0085  1.38232  1.30534  1.19134   

                             isHoliday  weekday  
id_road time     date                            
275911  00:00:00 2019-01-02        0.0        2  
                 2019-01-03        0.0        3  
                 2019-01-04        0.0        4  
                 2019-01-05        1.0        5  
                 2019-01-06        1.0        6  
...                                ...      ...  
276738  23:50:00 2019-12-16        0.0        0  
                 2019-12-17        0.0        1  
                 2019-12-18        0.0        2  
                 2019-12-19        0.0        3  
                 2019-12-20        0.0        4  

[296333 rows x 17 columns]

In [4]:
# 随机森林预测
for name, group in df2.groupby('id_road'):
    group.sort_values('time', inplace=True)
    group.set_index('time', inplace=True)
    group.index = group.index.time
    for i in range(6, len(group), 9):
        df = df1.loc[name, group.index[i]].values
        model = RandomForestRegressor()
        model.fit(df[:, :12], df[:, 12:15])
        group.iloc[i:i+3, 2] = model.predict(group.iloc[i-6:i, [2, 3]].values.T.reshape(1, -1)).ravel()
        df3 = df3.append(group.iloc[i:i+3, [0, 2]])
df3.reset_index(drop=True, inplace=True)
df3['id_sample'] = df3['id_sample'].astype(int)
df3.sort_values('id_sample', inplace=True)
# df3.to_csv('submit11.csv', index=False)
df3

,id_sample,TTI
504,0,1.081019
505,1,1.073067
506,2,1.093142
507,3,1.866500
508,4,2.061127
...,...,...
3019,3019,1.610964
3020,3020,1.633736
3021,3021,1.645765
3022,3022,1.553734


In [ ]:
# XGBoost
params = {}
for name, group in df2.groupby('id_road'):
    group.sort_values('time', inplace=True)
    group.set_index('time', inplace=True)
    group.index = group.index.time
    for i in range(6, len(group), 9):
        df = df1.loc[name, group.index[i]].values
        dtest = xgb.DMatrix(group.iloc[i-6:i, [2, 3]].values.T.reshape(1, -1))
        dtrain = xgb.DMatrix(df[:, :12], label=df[:, 12])
        bst = xgb.train(params, dtrain);
        t1 = bst.predict(dtest)[0]
        dtrain = xgb.DMatrix(df[:, :12], label=df[:, 13])
        bst = xgb.train(params, dtrain);
        t2 = bst.predict(dtest)[0]
        dtrain = xgb.DMatrix(df[:, :12], label=df[:, 14])
        bst = xgb.train(params, dtrain);
        t3 = bst.predict(dtest)[0]
        group.iloc[i:i+3, 2] = [t1, t2, t3]
        df3 = df3.append(group.iloc[i:i+3, [0, 2]])
df3.reset_index(drop=True, inplace=True)
df3['id_sample'] = df3['id_sample'].astype(int)
df3.sort_values('id_sample', inplace=True)
# df3.to_csv('submit11.csv', index=False)
df3

In [6]:
df4 = pd.read_csv('train_TTI_6plus3.csv')
model = KNeighborsRegressor(46)
model.fit(df4.loc[:, '1':'6'], df4.loc[:, '7':'9'])
df5 = pd.concat([pd.read_csv('toPredict_noLabel.csv'), pd.read_csv('toPredict_train_TTI.csv')])
df5['time'] = pd.to_datetime(df5['time'])
df6 = pd.DataFrame(columns=['id_sample', 'TTI'])
for name, group in df5.groupby('id_road'):
    group.sort_values('time', inplace=True)
    group.set_index('time', inplace=True)
    group.index = group.index.time
    for i in range(6, len(group), 9):
        group.iloc[i:i+3, 2] = model.predict(group.iloc[i-6:i, 2].values.reshape(1, -1)).ravel()
        df6 = df6.append(group.iloc[i:i+3, [0, 2]])
df6['id_sample'] = df6['id_sample'].astype(int)
df6.sort_values('id_sample', inplace=True)
df6.reset_index(drop=True, inplace=True)
# df6.to_csv('submit13.csv', index=False)
df6

,id_sample,TTI
0,0,1.073613
1,1,1.081165
2,2,1.090378
3,3,1.625202
4,4,1.641998
...,...,...
3019,3019,1.618468
3020,3020,1.619366
3021,3021,1.588052
3022,3022,1.582073
